In [2]:
pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
# --- 1. 目标网站和基本设置 ---
URL = "https://ideas.repec.org/top/top.econdept.html"
# 设置请求头，模拟浏览器访问，避免被网站拦截
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [5]:
# --- 2. 发送请求，获取网页内容 ---
print("正在访问目标网站...")
try:
    response = requests.get(URL, headers=HEADERS)
    response.raise_for_status() # 如果请求失败 (比如 404), 会抛出异常
    print("网站访问成功！")
except requests.RequestException as e:
    print(f"访问网站失败: {e}")
    exit() # 如果网站访问不了，就退出程序

正在访问目标网站...
网站访问成功！


In [6]:
# --- 3. 解析网页，提取大学名称 ---
# 使用 BeautifulSoup 解析 HTML
soup = BeautifulSoup(response.text, 'html.parser')

# 创建一个空列表来存放所有大学的名字
all_universities = []

# 首先，找到包含排名的那个大表格，它的 id 是 'ranklist'
rank_table = soup.find('table', {'class': 'shorttop'})

if rank_table:
    print("已找到排名表格，正在提取大学名称...")
    # 找到表格中所有的行 <tr>
    # 我们要跳过第一行，因为那是表头
    rows = rank_table.find_all('tr')[1:] 
    
    for row in rows:
        # 每一行里有很多列 <td>，大学名称在第二列 (索引为 1)
        cols = row.find_all('td')
        if len(cols) > 1:
            # 提取第二列的文本，并用 .strip() 去掉前后的空格
            university_name_raw = cols[1].text.strip()
            
            # 清洗数据：名字通常是 "University Name - Department of..." 的格式
            # 我们只想要前半部分 "University Name"
            university_name_clean = university_name_raw.split(' - ')[0]
            
            all_universities.append(university_name_clean)
else:
    print("错误：在页面上没有找到 id 为 'ranklist' 的表格。")
    exit()

print(f"成功提取了 {len(all_universities)} 所大学机构的名称。")

已找到排名表格，正在提取大学名称...
成功提取了 338 所大学机构的名称。


In [7]:
# --- 4. 按照你的要求选取30所大学 ---
# 10 from top 30, 10 from top 60, 10 from top 90
# 为了简单且符合要求，我们取 1-10名, 31-40名, 61-70名
# 注意列表索引是从0开始的
selected_universities = []
if len(all_universities) >= 90:
    group1 = all_universities[0:10]    # Top 10 (排名 1-10)
    group2 = all_universities[30:40]   # (排名 31-40)
    group3 = all_universities[60:70]   # (排名 61-70)
    selected_universities = group1 + group2 + group3
    print("已按要求选取30所大学。")
else:
    print("警告：提取到的大学数量不足90所，将使用能获取到的最多的大学进行分组合并。")
    # 如果数量不够，就尽量取
    selected_universities = all_universities[:10] + all_universities[30:min(40, len(all_universities))] + all_universities[60:min(70, len(all_universities))]

已按要求选取30所大学。


In [8]:
# --- 5. 保存到 Excel 文件 ---
# 将列表转换为 pandas DataFrame
df_to_save = pd.DataFrame(selected_universities, columns=['UniversityName'])

# 保存为一个新的 Excel 文件，这样更安全，不会破坏你原来的 data.xlsx
output_filename = "universities_list.xlsx"
df_to_save.to_excel(output_filename, index=False)

print(f"\n🎉 成功！大学列表已保存到 '{output_filename}' 文件中。")
print("现在，请打开这个新文件，将大学名单复制到你的 `data.xlsx` 的 `Universities` 工作表中。")


🎉 成功！大学列表已保存到 'universities_list.xlsx' 文件中。
现在，请打开这个新文件，将大学名单复制到你的 `data.xlsx` 的 `Universities` 工作表中。


In [1]:
import os
print(os.getcwd())

C:\Users\31616\zPython_econ\Grad_Application_Generator\HW_School_Application
